# Document Extraction Daloopa


The objective is to demonstrate detection of tables in raw texxt to use targeted table extraction and lower the cost of using Textract.
Exmple for 1M pages per month containing 19% of pages with at leaqst 1 table. 

https://calculator.aws/#/estimate?id=18c806f2696f894da0c8967a1697f1120279f693


# Notebook Setup

In this step, we will import some necessary libraries that will be used throughout this notebook. 

In [3]:
!python -m pip install -q amazon-textract-response-parser --upgrade
!python -m pip install -q amazon-textract-caller --upgrade
!python -m pip install -q amazon-textract-prettyprinter==0.0.16
!python -m pip install -q amazon-textract-textractor --upgrade
%pip install pymupdf 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-textract-textractor 1.1.1 requires tabulate==0.9.*, but you have tabulate 0.8.10 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-textract-prettyprinter 0.0.16 requires tabulate==0.8.10, but you have tabulate 0.9.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import boto3
import botocore
import sagemaker
import pandas as pd
from IPython.display import Image, display, JSON, HTML
from textractcaller.t_call import call_textract, Textract_Features, call_textract_expense
from textractprettyprinter.t_pretty_print import convert_table_to_list
from trp import Document
import os


# variables
data_bucket = sagemaker.Session().default_bucket()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sagemaker.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3')
textract = boto3.client('textract', region_name=region)
comprehend=boto3.client('comprehend', region_name=region)
comprehendmedical=boto3.client('comprehendmedical', region_name=region)

SageMaker role is: arn:aws:iam::923866638787:role/service-role/AmazonSageMaker-ExecutionRole-20210225T090795
Default SageMaker Bucket: s3://sagemaker-us-east-1-923866638787


In [27]:
file="./Demo/Daloopa/21304113.pdf"
#file="./Demo/Daloopa/21964974.pdf"
#file="./Demo/Daloopa/23479970.pdf"
file_key=f"idp/textract/demo/{os.path.basename(file)}"

!aws s3 cp {file} s3://sagemaker-us-east-1-923866638787/{file_key} --only-show-errors

In [36]:
!pwd

/root/aws-ai-intelligent-document-processing


In [28]:

from IPython.display import IFrame
IFrame("aws-ai-intelligent-document-processing/"+file, width=1500, height=600)

---
Extract unstructured data with Amazon Textract 

Amazon Textract is an ML powered OCR service that is capable of detecting and extracting text from documents. Text data in the form of WORDS and LINES can be extracted from documents using Amazon Textract `DetectDocumentText` API. Let's extract the words and lines from the bank statement.

In [32]:
# Call Amazon Textract
response = call_textract(input_document="s3://sagemaker-us-east-1-923866638787/"+file_key)

text=""
# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        text=text+"\n"+item["Text"]
print (text)


Supplemental Financial and Operating Data
AMERICAN TOWER
December 31, 2020
CORPORATION
americantower.com
AMERICAN TOWER®
Q4 2020 SUPPLEMENTAL DISCLOSURES
CORPORATION
TABLE OF CONTENTS:
Page
Safe Harbor Statement
3
Corporate Information
4
Analyst Coverage
5
Common Stock Data
6
Dividend Policy and Stock Repurchase History
7
Section 1: Company & Portfolio Overview
Company Profile
9
U.S. Portfolio
10
International Portfolio
11
Historical Tower Count
12
Tenant and Property Interest Overview
13
Section 2: Historical Financial and Supplemental Data
Consolidated Balance Sheets
15
Consolidated Statements of Operations
16
Condensed Consolidated Statements of Cash Flows
17
Historical Reconciliations
18-19
Historical Supplemental Details
20
Illustrative Projections and Outlook Sensitivity Analysis
21
Section 3: Regional Tear Sheets
Segment Disclosures
23-26
U.S. & Canada Portfolio Information
27
U.S. & Canada Tear Sheet
28
Asia-Pacific Portfolio Information
29
Asia-Pacific Tear Sheet
30
Africa Po

---
# Page metrics



In [30]:
from collections import Counter


def containsatable(textcontent):
    counter = Counter(textcontent)
    mostcom = counter.most_common(15)
    countdigit=0
    for i in mostcom :
        if (i[0].isdigit()):
            countdigit=countdigit+1
    return(countdigit>=4)



In [31]:
from trp import Document
import math
doc = Document(response)
#print(doc.pages[0])
# Iterate over elements in the document
pagenum=0
pageswithtables=[]
for page in doc.pages:
    pagenum=pagenum+1
    text=''
    for line in page.lines:
        text=text+line.text
    if (containsatable(text)):
        pageswithtables.append(pagenum)

        
print (pageswithtables)
        
print (str(len(pageswithtables))+"/"+str(pagenum))
print (str(math.trunc(len(pageswithtables)/pagenum*100))+"%")
    


[12, 15, 16, 18, 19, 25, 26, 28, 30, 32, 34, 36, 39]
13/43
30%


In [8]:
%pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
from PyPDF2 import PdfReader, PdfWriter

pdf_file_path = file
file_base_name = pdf_file_path.replace('.pdf', '')

pdf = PdfReader(pdf_file_path)

pages = pageswithtables
pdfWriter = PdfWriter()

for page_num in pages:
    pdfWriter.add_page(pdf.pages[page_num])

pdf_out = open("temp.pdf", 'wb')
pdfWriter.write(pdf_out)
pdf_out.close()


In [21]:
!aws s3 cp temp.pdf s3://sagemaker-us-east-1-923866638787/daloopa/temp.pdf --only-show-errors

In [24]:
resp = call_textract(input_document="s3://sagemaker-us-east-1-923866638787/daloopa/temp.pdf", features=[Textract_Features.TABLES])
tdoc = Document(resp)
dfs = list()

for page in tdoc.pages:
    for table in page.tables:
        tab_list = convert_table_to_list(trp_table=table)
        print(tab_list)
        dfs.append(pd.DataFrame(tab_list))

df1 = dfs[0]
df2 = dfs[1]

[['', '', '46 ', '48 ', '51 ', '47 ', '49 '], ['', '40 ', '', '', 'mm ', '', ''], ['', 'Q3-20 ', 'Q4-20 ', 'Q1-21 ', 'Q2-21 ', 'Q3-21 ', 'Q4-21 '], ['Y/Y Growth ', '27% ', '35% ', '51% ', '38% ', '19% ', '5% '], ['FXN Y/Y Growth ', '24% ', '31% ', '44% ', '30% ', '17% ', '7% '], ['Share of Total TPV ', '16% ', '17% ', '17% ', '16% ', '15% ', '14% '], ['ex-eBay FXN Y/Y Growth ', '25% ', '29% ', '41% ', '28% ', '19% ', '14% ']]
[['(in millions, except %) ', 'Q3-20 ', 'Q4-20 ', 'Q1-21 ', 'Q2-21 ', 'Q3-21 ', 'Q4-21 '], ['Transaction Revenue ', '$5,076 ', '$5,682 ', '$5,621 ', '$5,797 ', '$5,607 ', '$6,377 '], ['Y/Y Growth ', '28% ', '25% ', '33% ', '17% ', '10% ', '12% '], ['Other Value Added Services (OVAS) ', '$383 ', '$434 ', '$412 ', '$441 ', '$575 ', '$541 '], ['Y/Y Growth ', '(9%) ', '2% ', '2% ', '40% ', '50% ', '25% '], ['Total Revenue ', '$5,459 ', '$6,116 ', '$6,033 ', '$6,238 ', '$6,182 ', '$6,918 '], ['Y/Y Growth ', '25% ', '23% ', '31% ', '19% ', '13% ', '13% '], ['FXN Y/Y Gro